In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

import numpy as np
import RENT

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import f1_score, precision_score, recall_score, matthews_corrcoef


from sklearn.datasets import load_breast_cancer

wisconsin = load_breast_cancer()
data = pd.DataFrame(wisconsin.data)
target = wisconsin.target

from sklearn.model_selection import train_test_split
original_train_data, original_test_data, original_train_labels, original_test_labels = \
train_test_split(data, target, random_state=0, shuffle=True)

In [2]:
run = True 
counter = 0
max_counts = 1
rs = 0
n_folds = 5

my_data = original_train_data.copy()
my_target = pd.DataFrame(original_train_labels.copy()) # must be DataFrame

In [3]:
# run RENT until counter reaches max_counts

while run==True:
    counter += 1
    # k-fold splits
    skf = StratifiedKFold(n_splits=n_folds, random_state = rs, shuffle=True)
    
    #for next RENT-run (if counter != max_count, use another random state to split the data)
    rs += 1
    
    # C and l1 and testsize-range parameters to try 
    my_C_params = [0.1, 1, 10]
    my_l1_params = [0.1, 0.5, 0.9]
    testsize_range = (0.25, 0.25)
    
    # store prediction scores of train and test data for each split in dataframe (end with a list of dataframes)
    fs_scores = list()
    for s in range(skf.get_n_splits()):
        fs_scores.append(pd.DataFrame(index=my_l1_params, columns=my_C_params))
    scores_pred = list()
    for s in range(skf.get_n_splits()):
        scores_pred.append(pd.DataFrame(index=my_l1_params, columns=my_C_params))
    
    # define all dicts where we want to store results
    incorrect_labels_dict = {}
    incorr_avg = {}
    incorr_max = {}
    incorr_min = {}
    count_features = {}
    union_features = {}
    intersection_features = {}
    score_prediction = {}
    models = {}
    
    # now we perform RENT feature selection
    for C in my_C_params:
        for l1 in my_l1_params:
            i = 0 # counter 
            
            incorr_labels, mod, scores, features = [], [], [], []
            df = pd.DataFrame(index=my_data.index)
            
            for train, test in skf.split(my_data, my_target):
                train_data = my_data.iloc[train,:]
                train_target = my_target.iloc[train,:]
                
                analysis = RENT.RENT_Classification(data=train_data,
                                                       target=train_target.iloc[:,0],
                                                       feat_names=train_data.columns,
                                                       C=[C],
                                                       l1_ratios=[l1],
                                                       parameter_selection=False,
                                                       poly='OFF',
                                                       testsize_range=testsize_range,
                                                       K=100,
                                                       method='logreg',
                                                       verbose=1)
                analysis.train()
                
                # perform feature selection
                sel_features = analysis.selectFeatures(tau_1=0.9, tau_2=0.9, tau_3=0.975)
                
                # store incorrect predictions
                incorrectness = analysis.summary_objects()
                
                # predict on test set to get prediction scores thereof
                sc = StandardScaler()
                train_sample = sc.fit_transform(train_data.iloc[:,sel_features])
                test_sample = sc.transform(my_data.iloc[test,sel_features])
                
                model = LR(penalty='none', max_iter=8000, solver='saga').fit(train_sample, train_target)
                score = matthews_corrcoef(my_target.iloc[test,:], model.predict(test_sample))
                
                # store results
                scores.append(score)
                mod.append(model)
                features.append(sel_features)
                incorr_labels.append(incorrectness.iloc[:,-1])
                
                df = pd.concat([df,incorrectness.iloc[:,-1]], axis=1)
                
                score_mat = analysis.get_enetParam_matrices()[0]
                
                fs_scores[i].loc[l1,C] = score_mat.values[0]
                scores_pred[i].loc[l1,C] = score
                
                i += 1
                
            # fill the dictionaries
            
            df.columns = ['fold{0}'.format(x+1) for x in range(skf.get_n_splits())]
            incorrect_labels_dict[(l1,C)] = df
            score_prediction[(l1,C)] = scores
            models[(l1,C)] = mod
            
            features_set = []
            for f in features:
                features_set.append(set(f))
                
            #union and intersection of all features across the k folds
            features_union = list(set.union(*features_set))
            features_intersection = list(set.intersection(*features_set)) 
            
            # count how often each feature was selected across the k folds (0 to k)
            features_count = pd.Series(0, index=features_union)
            for f in features:
                features_count.loc[f] += 1
            
            union_features[(l1,C)] = features_union
            intersection_features[(l1,C)] = features_intersection
            count_features[(l1,C)] = features_count
            
            # store average, maximal and minimal  incor. predicitons
            incorr_labels_avg = pd.DataFrame(np.round(list(np.apply_along_axis(np.nanmean,1,df))),\
                         index=my_data.index)
            incorr_labels_avg.columns = ["nr incorr"]

            incorr_labels_max = pd.DataFrame(list(np.apply_along_axis(np.nanmax,1,df)),\
                         index=my_data.index)
            incorr_labels_max.columns = ["nr incorr"]
            
            incorr_labels_min = pd.DataFrame(list(np.apply_along_axis(np.nanmin,1,df)),\
                         index=my_data.index)
            incorr_labels_min.columns = ["nr incorr"]
            
            
            incorr_avg[(l1,C)] = incorr_labels_avg
            incorr_max[(l1,C)] = incorr_labels_max
            incorr_min[(l1,C)] = incorr_labels_min

            
            
    # now find the best l1 and C
    mat_pred = pd.DataFrame(0, index=my_l1_params, columns=my_C_params)
    for s in scores_pred:
        mat_pred = mat_pred + s
    mat_pred = mat_pred/len(scores_pred)

    mat_train = pd.DataFrame(0, index=my_l1_params, columns=my_C_params)
    for s in fs_scores:
        mat_train = mat_train + s
    mat_train = mat_train/len(fs_scores)

    best_row, best_col  =np.where(mat_pred == np.max(mat_pred.values))
    best_l1 = mat_pred.index[np.max(best_row)]
    best_C = mat_pred.columns[np.min(best_col)]     
   
    run=False # we stop after one run... additional code could be added to run multiple times. Then we would need to 
    # store the reduced data matrix and rerun it
    # fs_data = my_data.loc[:, union_features[(best_l1,best_C)]]
        

data dimension: (340, 30)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension (340,)
regularization parameters C: [0.1]
elastic net l1_ratios: [0.1]
number of models in ensemble: 100
classification method: logreg
verbose: 1
l1 =  0.1 C =  0.1 , TT split =  1
l1 =  0.1 C =  0.1 , TT split =  0
l1 =  0.1 C =  0.1l1 =  0.1 C =  0.1 , TT split =  5
 l1 =  0.1 C =  0.1 , TT split =  6
l1 =  0.1, TT split =  2
 C =  0.1l1 =  0.1 C =  0.1 , TT split =  7
 , TT split =  3
l1 =  0.1 C =  0.1 , TT split =  4
l1 =  0.1 C =  0.1 , TT split =  8
l1 =  0.1 C =  0.1 , TT split =  9
l1 =  0.1 C =  0.1 , TT split =  10
l1 =  0.1 C =  0.1 , TT split =  11
l1 =  0.1 C =  0.1 , TT split =  14
l1 =  0.1 C =  0.1 , TT split =  13
l1 = l1 =  0.1 C =  0.1 , TT split =  12
 0.1 C =  0.1 , TT split =  15
l1 =  0.1 C =  0.1 , TT split =  16
l1 =  0.1 C =  0.1 , TT split =  17
l1 = l1 =  0.1  l1 =  0.1 C =  0.1 C =  0.1 , TT split =  20, TT split = 0.1 C =  0.1 
 19
, TT split =  18
l1 =  l1 = 0.1l

C:\NMBU\Miniconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


data dimension: (341, 30)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension (341,)
regularization parameters C: [0.1]
elastic net l1_ratios: [0.1]
number of models in ensemble: 100
classification method: logreg
verbose: 1
l1 =  0.1 C =  0.1 , TT split =  l1 =  0.1 C =  0.1 , TT split =  0
2
l1 = l1 =  0.1 C =  0.1 , TT split =  1
l1 =  0.1 l1 =  0.1 C = l1 =  0.1   0.1 , TT split =  C = C =  0.1 , TT split =  3
 0.1 C =  0.1 , TT split =  4
0.1 , TT split =  56

l1 =  0.1 C =  0.1 , TT split =  7
l1 =  0.1l1 =  C =  0.1 , TT split =  8
 0.1 C =  0.1 , TT split =  9
l1 =  0.1 C =  0.1 , TT split =  10
l1 =  0.1 C =  0.1l1 =  0.1  , TT split =  11
C =  0.1 , TT split =  14l1 = l1 =  0.1 C =  
 0.1 0.1 , TT split =  12
C =  0.1l1 =   0.1 C =  0.1 , TT split =  13
, TT split =  15
l1 =  0.1 C =  0.1 , TT split =  16
l1 =  0.1 C =  0.1 , TT split =  17
l1 =  0.1 C =  0.1 , TT split =  19
l1 =  0.1 C =  0.1 , TT split =  18
l1 =  0.1 C =  0.1 , TT split =  l1 =  0.1 C =  0.1

C:\NMBU\Miniconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


data dimension: (341, 30)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension (341,)
regularization parameters C: [0.1]
elastic net l1_ratios: [0.1]
number of models in ensemble: 100
classification method: logreg
verbose: 1
l1 =  0.1 C =  0.1 , TT split =  l1 =  0.1 C =  0.1 , TT split =  l1 =  0.1 C =  0.11
 , TT split =  4
l1 =  0.1 C =  0.1 , TT split =  2
l1 =  0.1 C =  0.1 , TT split =  75

l1 =  0.1 C =  0.1 , TT split =  0
l1 =  0.1 C =  0.1 , TT split =  6
l1 =  0.1 C =  0.1 , TT split =  3
l1 =  0.1 C =  0.1 , TT split =  8
l1 =  0.1 C =  0.1 , TT split =  9
l1 =  l1 =  0.1 C =  0.10.1 C =  l1 = l1 =  0.1 , TT split =  13
, TT split =  0.1 C =  0.1  12
 0.1 C =  0.1 , TT split =  10, TT split =  
11
l1 =  0.1 C =  l1 = 0.1 , TT split =  14
 0.1 C =  0.1 , TT split =  15
l1 =  0.1 C =  0.1 , TT split =  16
l1 =  0.1 C =  0.1 , TT split =  17
l1 = l1 =  0.1  C =  0.1 , TT split =  180.1 C =  
0.1l1 =  , TT split = l1 =   0.1 0.1 C =  0.1 , TT split =  20
l1 =  0.1

...

C:\NMBU\Miniconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


data dimension: (341, 30)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension (341,)
regularization parameters C: [10]
elastic net l1_ratios: [0.9]
number of models in ensemble: 100
classification method: logreg
verbose: 1
l1 =  l1 =  0.90.9 C =  10 , TT split =  0
l1 =  0.9 C =  l1 =  0.9 C =  10 10 , TT split = C = l1 =  0.9 C =   , TT split =  6
 10 , TT split =  1
l1 =  0.9 C =  10  3
, TT split = l1 =  0.9 5
 C = 10 , TT split =  4
 10 , TT split =  2
l1 =  0.9 C =  10 , TT split =  7
l1 =  0.9 C =  10 , TT split =  8
l1 =  0.9 C =  10 , TT split =  9
l1 =  0.9 C =  10 , TT split =  10
l1 =  0.9 C =  10 , TT split =  11
l1 =  0.9 C =  10 , TT split =  12
l1 =  0.9 C =  10 , TT split =  13
l1 =  0.9 C =  10 , TT split =  14
l1 =  0.9 C =  10 , TT split =  15
l1 =  0.9 C =  10 , TT split =  16
l1 =  0.9 C =  10 , TT split =  17
l1 =  0.9 C =  10 , TT split =  18
l1 =  0.9 C =  10 , TT split =  19
l1 =  0.9 C =  10 , TT split =  20
l1 =  0.9 C =  10 , TT split =  21
l1

C:\NMBU\Miniconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


data dimension: (341, 30)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension (341,)
regularization parameters C: [10]
elastic net l1_ratios: [0.9]
number of models in ensemble: 100
classification method: logreg
verbose: 1
l1 =  0.9 C =  10 , TT split =  0
l1 =  0.9 C =  10 , TT split =  1
l1 =  0.9 C =  10 , TT split =  2
l1 =  0.9 C = l1 =  0.9 C =  10 , TT split =  10 , TT split =  4
 3
l1 = l1 =  0.9 C =  10 0.9 C =  10 , TT split =  7 , TT split =  5
l1 =  0.9 C = 
 10 , TT split =  6
l1 =  0.9 C =  10 , TT split =  8
l1 =  0.9 C =  10 , TT split =  9
l1 =  0.9 C =  10 , TT split =  10
l1 =  0.9 C =  10 , TT split =  11
l1 =  0.9 C =  10 , TT split =  12
l1 =  0.9 C =  10 , TT split =  13
l1 =  0.9 C =  10 , TT split =  14
l1 =  0.9 C =  10 , TT split =  15
l1 =  0.9 C =  10 , TT split =  16
l1 =  0.9 C =  10 , TT split =  17
l1 =  0.9 C =  10 , TT split =  18
l1 =  0.9 C =  10 , TT split =  19
l1 =  0.9 C =  10 , TT split =  20
l1 =  0.9 C =  10 , TT split =  21
l1

C:\NMBU\Miniconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [4]:
best_l1

0.1

In [5]:
best_C

10.0

In [6]:
# for the best parameter combination: incorrect prediction % across all folds
incorrect_labels_dict[(best_l1, best_C)]

,fold1,fold2,fold3,fold4,fold5
0,NaN,0.000000,0.000000,0.000000,0.000000
2,NaN,0.000000,0.000000,0.000000,0.000000
3,0.000000,NaN,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,NaN
5,0.000000,0.000000,0.000000,NaN,0.000000
6,0.000000,0.000000,0.000000,NaN,0.000000
7,NaN,0.000000,0.000000,0.000000,0.000000
8,NaN,0.000000,0.000000,0.000000,0.000000
9,0.000000,NaN,0.000000,0.000000,0.000000
11,0.000000,NaN,0.000000,0.000000,0.000000


In [7]:
# average prediction error over all folds
incorr_avg[(best_l1, best_C)]

,nr incorr
293,0.0
332,0.0
565,0.0
278,0.0
489,0.0
346,0.0
357,0.0
355,0.0
112,0.0
68,0.0


In [8]:
# count how often a feature was selected through the k folds (left column:feature, right column:count)
count_features[(best_l1, best_C)]

0     1
1     4
2     1
3     1
5     2
6     5
7     5
8     4
10    5
11    1
12    5
13    5
15    5
17    1
18    2
19    5
20    5
21    5
22    5
23    5
24    2
25    1
26    5
27    5
28    1
29    5
dtype: int64

In [9]:
# features that were selected in at least one fold (all with count >=1)
union_features[(best_l1, best_C)]

[0,
 1,
 2,
 3,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29]

In [10]:
# intersection of selected features ( all features with count k - selected in each fold)
intersection_features[(best_l1, best_C)]

[6, 7, 10, 12, 13, 15, 19, 20, 21, 22, 23, 26, 27, 29]